In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import urllib.request
from tqdm import tqdm

In [3]:
knowlife_df = pd.read_csv('../data/knowlife_dump.tsv', sep='\t')
knowlife_df.head()

,leftfactentity,relation,rightfactentity,leftpatternentity,leftpatternphrase,pattern,rightpatternentity,rightpatternphrase,sentences,source
0,C0000098,createsRiskFor,C0332472,C0000098,MPP,induced,C0332472,fragmentation,"{""We further validated our findings in primary...",pub_med_medline
1,C0000194,alleviates,C0007785,C0000194,2 2 dipyridyl,reduce,C0007785,cerebral ischemia,"{""Previous studies have indicated that 2,2 ' -...",pub_med_medline
2,C0000194,alleviates,C0038525,C0000194,2 2 dipyridyl,reduce,C0038525,subarachnoid hemorrhage,"{""Previous studies have indicated that 2,2 ' -...",pub_med_medline
3,C0000194,alleviates,C0265110,C0000194,2 2 dipyridyl,reduce,C0265110,cerebral vasospasm,"{""Previous studies have indicated that 2,2 ' -...",pub_med_medline
4,C0000266,alleviates,C0030567,C0000266,Parlodel,treat symptoms,C0030567,Parkinson s disease,"{""Parlodel is also used to treat symptoms of P...",DrugsDotCom


In [4]:
set(knowlife_df['relation'])

{'aggravates',
 'alleviates',
 'brandName',
 'causes',
 'ci',
 'complicationOf',
 'createsRiskFor',
 'diagnoses',
 'heals',
 'interacts',
 'isChemClass',
 'isPharmaClass',
 'isSymptomOf',
 'isTherapeuticClass',
 nan,
 'notCauses',
 'notCreatesRiskFor',
 'notHeals',
 'notReducesRiskFor',
 'observedIn',
 'reducesRiskFor',
 'sideeffect'}

In [5]:
description_df = pd.read_csv('../data/D_ICD_DIAGNOSES.csv')
description_df.head()

,row_id,icd9_code,short_title,long_title
0,1,01716,Erythem nod tb-oth test,Erythema nodosum with hypersensitivity reactio...
1,2,01720,TB periph lymph-unspec,"Tuberculosis of peripheral lymph nodes, unspec..."
2,3,01721,TB periph lymph-no exam,"Tuberculosis of peripheral lymph nodes, bacter..."
3,4,01722,TB periph lymph-exam unk,"Tuberculosis of peripheral lymph nodes, bacter..."
4,5,01723,TB periph lymph-micro dx,"Tuberculosis of peripheral lymph nodes, tuberc..."


In [6]:
knowlife_df[knowlife_df['leftpatternphrase'].astype(str).apply(lambda x: x.lower()).str.contains('erythema nodosum')] #C0014743

,leftfactentity,relation,rightfactentity,leftpatternentity,leftpatternphrase,pattern,rightpatternentity,rightpatternphrase,sentences,source
119298,C0014743,causes,C0549099,C0014743,erythema nodosum,developed,C0549099,perforation,"{""During the next few days the patient develop...",pub_med_medline
119299,C0014743,observedIn,C0206131,C0014743,Erythema nodosum,inflammation,C0206131,fat cells,"{""Erythema nodosum -LRB- EN -RRB- -LRB- red no...",Wikipedia
450172,C0343467,complicationOf,C0023348,C0343467,Erythema nodosum leprosum,complication,C0023348,lepromatous leprosy,"{""Erythema nodosum leprosum -LRB- ENL -RRB- , ...",pub_med_medline


In [7]:
knowlife_df[knowlife_df['leftpatternphrase'].astype(str).apply(lambda x: x.lower()).str.contains('cholera')] #C0014743

,leftfactentity,relation,rightfactentity,leftpatternentity,leftpatternphrase,pattern,rightpatternentity,rightpatternphrase,sentences,source
28314,C0004093,sideeffect,C0008359,C0008359,Cholera Vaccine,Side Effects Unusual,C0004093,weakness,"{""Cholera Vaccine -LRB- Injection Route -RRB-""...",mayoclinic
35707,C0004722,reducesRiskFor,C0008354,C0008354,Cholera,prevention,C0004722,bananas,"{Cholera,Prevention,""such as bananas , oranges...",mayoclinic
52969,C0008354,causes,C0011175,C0008354,Cholera,causes,C0011175,dehydration,"{""Cholera causes severe diarrhea and dehydrati...",mayoclinic
52970,C0008354,causes,C0013473,C0008354,cholera,lead,C0013473,eating disorders,"{""Other diseases and disorders associated with...",Wikipedia
52971,C0008354,causes,C0239182,C0008354,Cholera,cause,C0239182,watery diarrhoea,"{""Cholera is a cause of acute watery diarrhoea...",pub_med_medline
52973,C0008354,causes,C1443924,C0008354,Cholera,causes,C1443924,severe diarrhea,"{""Cholera causes severe diarrhea and dehydrati...",mayoclinic
52974,C0008354,createsRiskFor,C0011991,C0008354,cholera,induced,C0011991,diarrhea,"{""This will replace the water and electrolyte ...",Wikipedia
52975,C0008354,createsRiskFor,C2825086,C0008354,cholera,likely develop,C2825086,blood types,"{""For reasons that are n't entirely clear , pe...",mayoclinic
52976,C0008356,createsRiskFor,C3662483,C0008356,cholera toxin,induces,C3662483,allergic sensitization,"{""The mucosal adjuvant cholera toxin induces a...",pub_med_medline
52979,C0008359,notReducesRiskFor,C0242781,C0008359,cholera vaccine,not prevent,C0242781,transmission infection,"{""Use of cholera vaccine does not prevent tran...",RxList


In [8]:
knowlife_df[knowlife_df['rightfactentity'] == 'C0002438']['sentences'].iloc[1]

'{"Entamoeba histolytica is the etiologic agent for amoebiasis ."}'

In [9]:
icd9data_base_url = 'http://www.icd9data.com'
icd9data_url = 'http://www.icd9data.com/2015/Volume1/default.htm'
icd9data_response = urllib.request.urlopen(icd9data_url)
icd9data_soup = BeautifulSoup(icd9data_response, 'html.parser', from_encoding=icd9data_response.info().get_param('charset'))

In [10]:
import time

def open_url(url):
    time.sleep(3)
    request = urllib.request.Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    response = urllib.request.urlopen(request)
    return BeautifulSoup(response, 'html.parser', from_encoding=response.info().get_param('charset'))

def open_url_retries(url, max_retries=10):
    try:
        return open_url(url)
    except urllib.error.HTTPError as e:
        print('Whooops!', e)
        if max_retries < 0:
            raise e
        else:
            time.sleep(60)
            return open_url_retries(url, max_retries-1)

def get_final_hierarchy_from(parent_url, parent_name, parent_code):
    print('Querying', parent_url)
    soup = open_url_retries(parent_url)

    hierarchy_df = pd.DataFrame(columns=['parent_url', 'parent_name', 'parent_code', 'child_url', 'child_name', 'child_code'])
    definition_list = soup.find_all(class_='codeHierarchyUL')[0]
    for list_item in definition_list.find_all('li'):
        child_url = icd9data_base_url + list_item.a['href']
        child_name = list_item.find_all(class_='threeDigitCodeListDescription')[0].get_text()
        child_code = list_item.a.get_text()
        hierarchy_df = hierarchy_df.append({
            'parent_url': parent_url, 
            'parent_name': parent_name, 
            'parent_code': parent_code, 
            'child_url': child_url, 
            'child_name': child_name,
            'child_code': child_code,
        }, ignore_index=True)
    return hierarchy_df

def get_hierarchy_from(parent_url, parent_name, parent_code):
    print('Querying', parent_url)
    soup = open_url_retries(parent_url)

    hierarchy_df = pd.DataFrame(columns=['parent_url', 'parent_name', 'parent_code', 'child_url', 'child_name', 'child_code'])
    definition_list = soup.find_all(class_='definitionList')[0]
    for list_item in definition_list.find_all('li'):
        child_url = icd9data_base_url + list_item.a['href']
        child_text = list_item.get_text()
        child_code = child_text.split(' ')[0]
        child_name = ' '.join(child_text.split(' ')[1:])
        hierarchy_df = hierarchy_df.append({
            'parent_url': parent_url, 
            'parent_name': parent_name, 
            'parent_code': parent_code, 
            'child_url': child_url, 
            'child_name': child_name,
            'child_code': child_code,
        }, ignore_index=True)
        if '-' in child_code:
            hierarchy_df = hierarchy_df.append(get_hierarchy_from(child_url, child_name, child_code), ignore_index=True)
        else:
            hierarchy_df = hierarchy_df.append(get_final_hierarchy_from(child_url, child_name, child_code), ignore_index=True)

    
    return hierarchy_df

icd9 = pd.read_csv('icd9.csv') #get_hierarchy_from(icd9data_url, 'root', -1)
icd9.head()

,Unnamed: 0,parent_url,parent_name,parent_code,child_url,child_name,child_code
0,0,http://www.icd9data.com/2015/Volume1/default.htm,root,-1,http://www.icd9data.com/2015/Volume1/001-139/d...,Infectious And Parasitic Diseases,001-139
1,1,http://www.icd9data.com/2015/Volume1/001-139/d...,Infectious And Parasitic Diseases,001-139,http://www.icd9data.com/2015/Volume1/001-139/0...,Intestinal Infectious Diseases,001-009
2,2,http://www.icd9data.com/2015/Volume1/001-139/0...,Intestinal Infectious Diseases,001-009,http://www.icd9data.com/2015/Volume1/001-139/0...,Cholera,001
3,3,http://www.icd9data.com/2015/Volume1/001-139/0...,Cholera,001,http://www.icd9data.com/2015/Volume1/001-139/0...,Cholera,001
4,4,http://www.icd9data.com/2015/Volume1/001-139/0...,Cholera,001,http://www.icd9data.com/2015/Volume1/001-139/0...,Cholera due to vibrio cholerae,001.0


In [11]:
knowlife_entities = pd.DataFrame(columns=['pattern_id', 'phrase'])
knowlife_entities = knowlife_entities.append(pd.DataFrame({
    'pattern_id': knowlife_df['leftpatternentity'].tolist(),
    'phrase': knowlife_df['leftpatternphrase'].apply(lambda x: str(x).lower()).apply(lambda x: ' '.join([(w[0:len(w)-1] if (len(w) > 1 and w[len(w)-1] == 's') else w) for w in x.split(' ')])).tolist(),
}), ignore_index=True)
knowlife_entities = knowlife_entities.append(pd.DataFrame({
    'pattern_id': knowlife_df['rightpatternentity'].tolist(),
    'phrase': knowlife_df['rightpatternphrase'].apply(lambda x: str(x).lower()).apply(lambda x: ' '.join([(w[0:len(w)-1] if (len(w) > 1 and w[len(w)-1] == 's') else w) for w in x.split(' ')])).tolist(),
}), ignore_index=True)
knowlife_entities = knowlife_entities.drop_duplicates()
knowlife_entities = knowlife_entities.dropna()
knowlife_entities = knowlife_entities.reset_index(drop=True)
knowlife_entities

,pattern_id,phrase
0,C0000098,mpp
1,C0000194,2 2 dipyridyl
2,C0000266,parlodel
3,C0020823,ifosfamide
4,C0721031,ifex
...,...,...
44202,C3653931,other vaccine
44203,C0745413,ischemic attack
44204,C0026820,contractile
44205,C3662017,combativenes


In [13]:
knowlife_ids = set(knowlife_entities['pattern_id'])
len(knowlife_ids)

35208

In [14]:
def open_url(url):
    time.sleep(3)
    request = urllib.request.Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    response = urllib.request.urlopen(request)
    return BeautifulSoup(response, 'html5lib', from_encoding=response.info().get_param('charset'))

def open_url_retries(url, max_retries=10):
    try:
        return open_url(url)
    except urllib.error.HTTPError as e:
        print('Whooops!', e)
        if max_retries < 0:
            raise e
        else:
            time.sleep(60)
            return open_url_retries(url, max_retries-1)

def get_synonyms_for_id(id):
    url = 'http://knowlife.mpi-inf.mpg.de/entityListing.html?id=' + str(id)
    print('Querying', url)
    soup = open_url_retries(url)

    synonym_df = pd.DataFrame(columns=['code', 'synonyms', 'vocabs'])
    definition_list = soup.find_all('ul', id='syns')[0]
    synonyms = []
    for list_item in definition_list.find_all('li'):
        synonyms.append(list_item.get_text())

    vocabs = None

    
    return hierarchy_df

#icd9 = #get_hierarchy_from(icd9data_url, 'root', -1)
icd9.head()

,Unnamed: 0,parent_url,parent_name,parent_code,child_url,child_name,child_code
0,0,http://www.icd9data.com/2015/Volume1/default.htm,root,-1,http://www.icd9data.com/2015/Volume1/001-139/d...,Infectious And Parasitic Diseases,001-139
1,1,http://www.icd9data.com/2015/Volume1/001-139/d...,Infectious And Parasitic Diseases,001-139,http://www.icd9data.com/2015/Volume1/001-139/0...,Intestinal Infectious Diseases,001-009
2,2,http://www.icd9data.com/2015/Volume1/001-139/0...,Intestinal Infectious Diseases,001-009,http://www.icd9data.com/2015/Volume1/001-139/0...,Cholera,001
3,3,http://www.icd9data.com/2015/Volume1/001-139/0...,Cholera,001,http://www.icd9data.com/2015/Volume1/001-139/0...,Cholera,001
4,4,http://www.icd9data.com/2015/Volume1/001-139/0...,Cholera,001,http://www.icd9data.com/2015/Volume1/001-139/0...,Cholera due to vibrio cholerae,001.0


In [15]:
import requests
url = 'http://knowlife.mpi-inf.mpg.de/entityListing.html?id=' + str('C0178238')
response = requests.get(url, timeout=(10, 10))
#request = urllib.request.Request(url, headers={'User-Agent': 'Mozilla/5.0'})
#response = urllib.request.urlopen(request, timeout=10)
soup = BeautifulSoup(response.content, 'html5lib')#, from_encoding=response.info().get_param('charset'))

#synonym_df = pd.DataFrame(columns=['code', 'synonyms', 'vocabs'])
#definition_list = soup.find('ul', id='syns')
#synonyms = []
#for list_item in definition_list.find_all('li'):
#    synonyms.append(list_item.get_text())
soup

<!DOCTYPE html>
<html><head>
<title>KnowLife - One-Stop Health Portal</title>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
 <meta content="yes" name="apple-mobile-web-app-capable"/>
 
<script src="https://code.jquery.com/jquery-2.1.0.js"></script>
<script src="https://code.jquery.com/mobile/1.4.1/jquery.mobile-1.4.1.min.js"></script>

<script charset="utf-8" src="js/navigationScripts.js" type="text/javascript"></script>
<script charset="utf-8" src="js/jquery.scrollTo.js" type="text/javascript"></script>
<script src="js/elasticsearch/elasticsearch.jquery.min.js"></script>
<script src="js/entListingScripts.js"></script>


<link href="https://code.jquery.com/mobile/1.4.1/jquery.mobile-1.4.1.min.css" rel="stylesheet"/>


<link href="css/other.css" media="screen" rel="stylesheet"/>

<style>
.ui-content .ui-listview, .ui-panel-inner>.ui-listview, .ui-listview-filter-inset {
	margin:0px;
}
</style>
</head>

<body>
<div data-role="page" id="entityListing">
<div data-ro

In [12]:
knowlife_entities[knowlife_entities['pattern_id'] == 'C0017536']

,pattern_id,phrase
12653,C0017536,giardiasi
12654,C0017536,giardia


In [17]:
icd9_entities[icd9_entities['code'] == '007.1']

,name,code
53,giardiasi,007.1


In [16]:
icd9_entities = pd.DataFrame()
icd9_entities['name'] = icd9['child_name'].apply(lambda x: str(x).lower()).apply(lambda x: ' '.join([(w[0:len(w)-1] if (len(w) > 1 and w[len(w)-1] == 's') else w) for w in x.split(' ')]))
icd9_entities['code'] = icd9['child_code']
icd9_entities = icd9_entities.drop_duplicates().reset_index(drop=True)
icd9_entities

,name,code
0,infectiou and parasitic disease,001-139
1,intestinal infectiou disease,001-009
2,cholera,001
3,cholera due to vibrio cholerae,001.0
4,cholera due to vibrio cholerae el tor,001.1
...,...,...
17721,injury due to other war operation but occurrin...,E998.8
17722,injury due to unspecified war operation but oc...,E998.9
17723,late effect of injury due to war operation and...,E999
17724,late effect of injury due to war operation,E999.0


In [25]:
merged = pd.merge(icd9_entities, knowlife_entities, left_on='name', right_on='phrase', how='left')
print('% non-matched codes', len(merged[merged['pattern_id'].isna()]) / len(merged))
print('% non-matched codes that are not other', len(merged[merged['pattern_id'].isna() & ~merged['name'].str.contains('other')]) / len(merged))
merged[merged['pattern_id'].isna()]

% non-matched codes 0.9428153317949006
% non-matched codes that are not other 0.7117690099622896


,name,code,pattern_id,phrase
0,infectiou and parasitic disease,001-139,NaN,NaN
1,intestinal infectiou disease,001-009,NaN,NaN
3,cholera due to vibrio cholerae,001.0,NaN,NaN
4,cholera due to vibrio cholerae el tor,001.1,NaN,NaN
5,"cholera, unspecified",001.9,NaN,NaN
...,...,...,...,...
17762,injury due to other war operation but occurrin...,E998.8,NaN,NaN
17763,injury due to unspecified war operation but oc...,E998.9,NaN,NaN
17764,late effect of injury due to war operation and...,E999,NaN,NaN
17765,late effect of injury due to war operation,E999.0,NaN,NaN


In [26]:
merged2 = pd.merge(icd9_entities, knowlife_entities, left_on='name', right_on='phrase', how='right')
print('% non-matched knowlife entities', len(merged2[merged2['code'].isna()]) / len(merged2))
merged2[merged2['code'].isna()]

% non-matched knowlife entities 0.9770198136252601


,name,code,pattern_id,phrase
0,NaN,NaN,C0000098,mpp
1,NaN,NaN,C0000194,2 2 dipyridyl
2,NaN,NaN,C0000266,parlodel
3,NaN,NaN,C0020823,ifosfamide
4,NaN,NaN,C0721031,ifex
...,...,...,...,...
44207,NaN,NaN,C3653931,other vaccine
44208,NaN,NaN,C0745413,ischemic attack
44209,NaN,NaN,C0026820,contractile
44210,NaN,NaN,C3662017,combativenes


In [27]:
icd9_knowlife_mapping_df = merged.dropna()[['code', 'pattern_id', 'name']].drop_duplicates().reset_index(drop=True)
icd9_knowlife_mapping_df

,code,pattern_id,name
0,001,C0008354,cholera
1,002.0,C0041466,typhoid fever
2,004,C0013371,shigellosi
3,004.0,C0036956,shigella dysenteriae
4,004.0,C0302358,shigella dysenteriae
...,...,...,...
1011,E926,C0455620,exposure to radiation
1012,E928.1,C0700522,exposure to noise
1013,E928.2,C0455941,vibration
1014,E928.2,C0677519,vibration


In [28]:
icd9[icd9['parent_code'] == '006'] #amoebiasis vs Amebiasis

,Unnamed: 0,parent_url,parent_name,parent_code,child_url,child_name,child_code
47,47,http://www.icd9data.com/2015/Volume1/001-139/0...,Amebiasis,006,http://www.icd9data.com/2015/Volume1/001-139/0...,Amebiasis,006
48,48,http://www.icd9data.com/2015/Volume1/001-139/0...,Amebiasis,006,http://www.icd9data.com/2015/Volume1/001-139/0...,Acute amebic dysentery without mention of abscess,006.0
49,49,http://www.icd9data.com/2015/Volume1/001-139/0...,Amebiasis,006,http://www.icd9data.com/2015/Volume1/001-139/0...,Chronic intestinal amebiasis without mention o...,006.1
50,50,http://www.icd9data.com/2015/Volume1/001-139/0...,Amebiasis,006,http://www.icd9data.com/2015/Volume1/001-139/0...,Amebic nondysenteric colitis,006.2
51,51,http://www.icd9data.com/2015/Volume1/001-139/0...,Amebiasis,006,http://www.icd9data.com/2015/Volume1/001-139/0...,Amebic liver abscess,006.3
52,52,http://www.icd9data.com/2015/Volume1/001-139/0...,Amebiasis,006,http://www.icd9data.com/2015/Volume1/001-139/0...,Amebic lung abscess,006.4
53,53,http://www.icd9data.com/2015/Volume1/001-139/0...,Amebiasis,006,http://www.icd9data.com/2015/Volume1/001-139/0...,Amebic brain abscess,006.5
54,54,http://www.icd9data.com/2015/Volume1/001-139/0...,Amebiasis,006,http://www.icd9data.com/2015/Volume1/001-139/0...,Amebic skin ulceration,006.6
55,55,http://www.icd9data.com/2015/Volume1/001-139/0...,Amebiasis,006,http://www.icd9data.com/2015/Volume1/001-139/0...,Amebic infection of other sites,006.8
56,56,http://www.icd9data.com/2015/Volume1/001-139/0...,Amebiasis,006,http://www.icd9data.com/2015/Volume1/001-139/0...,"Amebiasis, unspecified",006.9


In [34]:
icd9_three_digits = icd9[~icd9['child_code'].str.contains('-')]
icd9_three_digits = icd9_three_digits[~icd9_three_digits['child_code'].str.contains('\.')]

icd9_knowlife_mapping = {}
for _, row in icd9.iterrows():
    parent_maps = set()
    child_maps = set()
    if row['parent_code'] in icd9_knowlife_mapping_df['code'].tolist():
        rel_df = icd9_knowlife_mapping_df[icd9_knowlife_mapping_df['code'] == row['parent_code']]
        parent_maps = set(rel_df['pattern_id'].tolist())
    if row['child_code'] in icd9_knowlife_mapping_df['code'].tolist():
        rel_df = icd9_knowlife_mapping_df[icd9_knowlife_mapping_df['code'] == row['child_code']]
        child_maps = set(rel_df['pattern_id'].tolist())
    icd9_knowlife_mapping[row['parent_code']] = parent_maps
    icd9_knowlife_mapping[row['child_code']] = child_maps

for _, row in icd9.iterrows():
    #if len(icd9_knowlife_mapping[row['child_code']]) == 0:
    icd9_knowlife_mapping[row['child_code']].update(icd9_knowlife_mapping[row['parent_code']])

[(x,y) for (x,y) in icd9_knowlife_mapping.items() if x in set(icd9_three_digits['child_code'])]

[('001', {'C0008354'}),
 ('002', set()),
 ('003', set()),
 ('004', {'C0013371'}),
 ('005', set()),
 ('006', {'C0013370'}),
 ('007', set()),
 ('008', set()),
 ('009', set()),
 ('010', set()),
 ('011', {'C0041327'}),
 ('012', set()),
 ('013', set()),
 ('014', set()),
 ('015', set()),
 ('016', set()),
 ('017', set()),
 ('018', {'C0041321'}),
 ('020', {'C0032064'}),
 ('021', {'C0041351'}),
 ('022', {'C0003175', 'C0004589', 'C1254288'}),
 ('023', {'C0006309'}),
 ('024', {'C0017589'}),
 ('025', {'C0025229'}),
 ('026', set()),
 ('027', set()),
 ('030', {'C0023343'}),
 ('031', set()),
 ('032', {'C0012546'}),
 ('033', {'C0043167'}),
 ('034', set()),
 ('035', {'C0014733'}),
 ('036', {'C0025303'}),
 ('037', {'C0039614'}),
 ('038', {'C0036690'}),
 ('039', set()),
 ('040', set()),
 ('041', set()),
 ('042', set()),
 ('045', set()),
 ('046', set()),
 ('047', set()),
 ('048', set()),
 ('049', set()),
 ('050', {'C0037354', 'C0037356'}),
 ('051', set()),
 ('052', {'C0008049'}),
 ('053', {'C0019360', 'C0